# CONTENT

* **a. Basic FNN** <== UNDER CONSTRUCTION

# Basic FNN

### A. Code

In [586]:
import numpy as np

In [587]:
import theano
from theano import shared
import theano.tensor as T
from theano.tensor.nnet import sigmoid, softmax

In [588]:
class Layer1:
    
    def __init__(self, bShape, wShape, activation=sigmoid):
        assert len(bShape) == len(wShape) == 2 # bShape, wShape: (row, col) 2-tuples.
        self.b = shared(np.asarray(np.random.normal(loc=0.,scale=1.,size=bShape),
                                   dtype=theano.config.floatX), borrow=True)
        self.w = shared(np.asarray(np.random.normal(loc=0.,scale=np.sqrt(1./wShape[0]),size=wShape),
                                   dtype=theano.config.floatX), borrow=True)
        self.params = [self.b, self.w]
        self.activation = activation
    
    def activate(self, x):
        z = T.dot(self.w, x) + self.b
        return z if self.activation is None else self.activation(z)

In [589]:
class NNTheano1:
    
    def __init__(self, sizes, hiddenActivation=sigmoid, outputActivation=sigmoid):
        self.numLayers = len(sizes)
        assert self.numLayers >= 3 # must have at least 1 hidden layer.
        self.layers = [ Layer1((nOut,1),(nOut,nIn),hiddenActivation) 
                        for nIn,nOut in zip(sizes[:-2],sizes[1:]) ]
        self.layers.append(Layer1((sizes[-1],1),(sizes[-1],sizes[-2]),outputActivation))
        self.params = []
        for layer in self.layers:
            self.params += layer.params
        
    def predict(self, a): # basically fwd. feeding.
        for layer in self.layers:
            a = layer.activate(a)
        return a
    
    def cost(self, x, y):
        return T.sum((self.predict(x)-y)**2)
    
    def train(self, X, Y, lr):
        self.sgd(lr)
        for x,y in zip(X,Y):
            self.run(x,y)   
    
    def sgd(self, lr): # set up training symbolic graph.
        inpt = T.matrix()
        output = T.matrix()
        cost = self.cost(inpt, output)
        updates = []
        for param in self.params:
            updates.append((param, param-lr*T.grad(cost,param)))
                # momentum version:
                #  paramUpdate = shared(param.get_value()*0, broadcastable=param.broadcastable) 
                #  updates.append((param, param-lr*paramUpdate))
                #  updates.append((paramUpdate, .9*paramUpdate + .1*T.grad(cost,param)))
        self.run = theano.function([inpt, output], cost, updates=updates)
            # f is a training function.
    
#     def evaluate(self, X, Y):
#         assert self.run != None # make sure sgd has been run.
#         correct = 0
#         for x,y in zip(X,Y):
            
            


### B. Toy Example: Logic Gate OR

In [590]:
# E.G. OR
'''
x y | lor
0 0 | 0
0 1 | 1
1 0 | 1
1 1 | 1
'''
xy1, lor1 = np.asarray([[0],[0]]), np.asarray([[0]])
xy2, lor2 = np.asarray([[0],[1]]), np.asarray([[1]])
xy3, lor3 = np.asarray([[1],[0]]), np.asarray([[1]])
xy4, lor4 = np.asarray([[1],[1]]), np.asarray([[1]])

In [591]:
'''
fnn: [2, 1, 1]
'''
fnn = NNTheano1([2,1,1])

In [592]:
xy1.shape

(2, 1)

In [593]:
lor1.shape

(1, 1)

In [594]:
X = [xy1, xy2, xy3, xy4]
Y = [lor1, lor2, lor3, lor4]

In [595]:
%%time
fnn.sgd(3.)
for _ in range(100):
    fnn.train(X,Y,3.)

CPU times: user 23.6 s, sys: 299 ms, total: 23.9 s
Wall time: 23.9 s


In [596]:
print "OR Truth-Value Table"
print "0 0 |", "0" if fnn.predict(xy1).eval()[0][0] < .5 else "1"
print "0 1 |", "0" if fnn.predict(xy2).eval()[0][0] < .5 else "1"
print "1 0 |", "0" if fnn.predict(xy3).eval()[0][0] < .5 else "1"
print "1 1 |", "0" if fnn.predict(xy4).eval()[0][0] < .5 else "1"

OR Truth-Value Table
0 0 | 0
0 1 | 1
1 0 | 1
1 1 | 1


### C. MNIST

In [597]:
import os
os.chdir("/Users/jacobsw/Desktop/IMPLEMENTATION_CAMP/CODE/BASIC_TOPICS/NN/DATA/neural-networks-and-deep-learning-master/src")
import mnist_loader # works when under '~/src' path.
data_train, data_dev, data_test = mnist_loader.load_data_wrapper()

In [598]:
X_train = [datum[0] for datum in data_train]
Y_train = [datum[1] for datum in data_train]

In [599]:
X_test = [datum[0] for datum in data_test]
Y_test = [datum[1] for datum in data_test]

In [600]:
fnn = NNTheano1([784, 30, 10], hiddenActivation=sigmoid, outputActivation=softmax)

In [579]:
%%time
fnn.train(X_train,Y_train,3.0)

CPU times: user 4.45 s, sys: 22.4 ms, total: 4.47 s
Wall time: 4.47 s


In [580]:
pred0 = fnn.predict(X_train[0])

In [583]:
Y_train[0]

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])

In [584]:
pred0.eval()

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]])

In [569]:
softmax(np.asarray([1,1,1,1],dtype=theano.config.floatX)).eval()

array([[ 0.25,  0.25,  0.25,  0.25]])

In [606]:
fnn.params[-2].eval()

array([[ 0.52157823],
       [ 0.11648892],
       [-0.30848819],
       [ 1.2731928 ],
       [-0.06909677],
       [ 0.78818831],
       [ 1.42269494],
       [-0.15164546],
       [-0.02529053],
       [ 0.24647496]])